In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:

import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime

1.8.1
0.8.0a0+e4e171a


In [3]:
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import glob
import random
from torch import nn
import torch.nn.functional as F

from catalyst import dl, utils
from catalyst.data.transforms import ToTensor

In [4]:
# from lark.learner import *
from lark.config import Config
# from lark.pann import *

In [11]:
cfg = Config(site='SSW', use_neptune=False)

In [12]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [20, 10, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': False,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.0001,
 'n_epochs': 10,
 'loss_fn': 'PANNsLoss',
 'optimizer': 'torch.optim.Adam',
 'scheduler': 'torch.optim.lr_scheduler.OneCycleLR',
 'model': 'Cnn14_DecisionLevelAtt',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  'haiwoo',
  'norcar',
  'norfli',
  'norwat',
  'ove

In [ ]:
# cfg.loss_fn = 'lark.Cnn14_DecisionLevelAtt.PANNsLoss'
# cfg.model = 'Cnn14_DecisionLevelAtt'
from lark.learner import Learner

lrn = Learner("pann", cfg)

In [60]:
lrn.learn()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   1 train loss: 0.577972 train f1: 0.042711 valid loss: 0.145022 valid f1: 0.210526


  0%|          | 0/336 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   9 train loss: 0.194136 train f1: 0.013310 valid loss: 0.121935 valid f1: 0.213765


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:  10 train loss: 0.191028 train f1: 0.019398 valid loss: 0.129105 valid f1: 0.213765


In [83]:
vdl = ValidDataset(cfg).loader
tdl = TrainDataset(cfg).loader

In [42]:
model = Cnn14_DecisionLevelAtt.load(cfg)

In [43]:
exp = Experiment(cfg)
exp.init(f"pann {datetime.now()}")

In [71]:
from lark.Cnn14_DecisionLevelAtt import PANNsLoss

In [68]:
class PANNsLoss(nn.Module):
    def __init__(self):
        super().__init__()

    @staticmethod
    def stabilize_input(cwo):
        cwo = torch.where(torch.isnan(cwo),
                          torch.zeros_like(cwo),
                          cwo)
        cwo = torch.where(torch.isinf(cwo),
                          torch.zeros_like(cwo),
                          cwo)
        cwo = cwo.clamp(0, 1)
        return cwo

    def forward(self, input, target):
        cwo = self.stabilize_input(input)
        return F.binary_cross_entropy(cwo, target)


In [84]:
eval('PANNsLoss')

lark.Cnn14_DecisionLevelAtt.PANNsLoss

In [86]:


cfg.loss_fn = 'Cnn14_DecisionLevelAtt.PANNsLoss'

loss_fn = cfg.instantiate_loss()
optimizer = cfg.instantiate_optimizer(model.parameters())
scheduler = cfg.instantiate_scheduler(optimizer)

NameError: name 'Cnn14_DecisionLevelAtt' is not defined

In [125]:
# n_samples = len(tdl)

# loss_fn = PANNsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr*10, steps_per_epoch=n_samples, epochs=n_epochs)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.01, last_epoch=-1)


In [45]:
lrn = Learner('pann', cfg)

NameError: name 'PANNsLoss' is not defined

In [126]:
model = model.cuda()
loss_fn = loss_fn.cuda()

In [127]:
# from tqdm import tqdm
import sys
from tqdm.notebook import tqdm

def tv_loop(dl, mode):
    if mode == 'train':
        model.train()
    else:
        model.eval()
        
    def inner_loop():
        nonlocal tot_loss
        nonlocal tot_score
        for x, y in pbar:
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            loss = loss_fn(pred, y)
            score = f1(y, pred, 0.5)
            
            pbar.set_description(f"{mode} loss: {loss:>8f} f1: {score:>8f}")
            exp.log_metric(mode, 'batch', 'loss', loss)
            exp.log_metric(mode, 'batch', 'f1', score)

            tot_loss += loss.item()
            tot_score += score
            if mode == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
        
    tot_loss = 0
    tot_score = 0
       
    with tqdm(dl, leave=False) as pbar:
        if mode == 'valid':
            with torch.no_grad():
                inner_loop()
        else:
            inner_loop()
                
    n_batches = len(dl)
    tot_loss /= n_batches
    tot_score /= n_batches
    return tot_loss, tot_score

In [128]:
with tqdm(range(n_epochs)) as pbar:
    for epoch in pbar:
        train_loss, train_score = tv_loop(tdl, 'train')
        valid_loss, valid_score = tv_loop(vdl, 'valid')
        msg = f"epoch: {epoch + 1:3d} train loss: {train_loss:>8f} train f1: {train_score:>8f} valid loss: {valid_loss:>8f} valid f1: {valid_score:>8f}"
        print(msg)
        exp.log_metric('train', 'epoch', 'loss', train_loss)
        exp.log_metric('valid', 'epoch', 'loss', valid_loss)
        exp.log_metric('train', 'epoch', 'f1', train_score)
        exp.log_metric('valid', 'epoch', 'f1', valid_score)


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   1 train loss: 0.525854 train f1: 0.036777 valid loss: 0.131547 valid f1: 0.210526


  0%|          | 0/336 [00:00<?, ?it/s]

...

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   4 train loss: 0.211320 train f1: 0.003988 valid loss: 0.116695 valid f1: 0.210526


  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
neptune.stop()

In [54]:
with tqdm(vdl, leave=False) as pbar:
    with torch.no_grad():
        for x, y in pbar:
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            score = f1(y, pred, 0.5)
            print(y)
            print(pred)
            print(score)

  0%|          | 0/38 [00:00<?, ?it/s]

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
  

KeyboardInterrupt: 